## Cell 1: Environment Setup

In [ ]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import importlib

# Import and reload scanner module (ensures latest code changes are used)
import sp500_scanner
importlib.reload(sp500_scanner)
from sp500_scanner import get_sp500_tickers, scan_sp500, filter_buy_signals, create_excel_output, create_pdf_report, cleanup_old_scans

print("✓ Imports complete")
print(f"Ready to scan S&P 500 as of {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✓ Imports complete
Ready to scan S&P 500 as of 2026-01-05 19:44


## Cell 2: Run Full S&P 500 Scan

**This will take 5-10 minutes to complete.**

Parameters:
- `daily_bars=60` - 60 days of daily data
- `weekly_bars=52` - 52 weeks of weekly data  
- `concurrency=15` - 15 parallel threads (adjust if needed)

In [2]:
# Run the scan
start_time = datetime.now()
print(f"🔍 Starting S&P 500 scan at {start_time.strftime('%H:%M:%S')}...\n")

results_df = scan_sp500(daily_bars=60, weekly_bars=52, concurrency=15)

elapsed = (datetime.now() - start_time).total_seconds()
print(f"\n✓ Scan completed in {elapsed:.1f} seconds")

🔍 Starting S&P 500 scan at 19:29:50...

Fetching S&P 500 ticker list...
✓ Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

🔍 Scanning 503 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 15 threads
✓ [9/503] AFL    -> FULL HOLD + ADD      $112.53
✓ [10/503] ABBV   -> FULL HOLD + ADD      $219.79
✓ [11/503] ALB    -> FULL HOLD + ADD      $147.80
✓ [18/503] ALL    -> FULL HOLD + ADD      $205.75
✓ [20/503] AMZN   -> FULL HOLD + ADD      $232.64
✓ [21/503] GOOG   -> FULL HOLD + ADD      $317.44
✓ [26/503] GOOGL  -> FULL HOLD + ADD      $316.89
✓ [28/503] AIG    -> FULL HOLD + ADD      $85.39
✓ [31/503] AXP    -> FULL HOLD + ADD      $382.22
✓ [32/503] AME    -> FULL HOLD + ADD      $211.77
✓ [33/503] AMGN   -> FULL HOLD + ADD      $319.33
✓ [34/503] APH    -> FULL HOLD + ADD      $140.17
✓ [35/503] ADI    -> FULL HOLD + ADD      $278.43
✓ [38/503] APP    -> FULL HOLD + ADD      $635.62
✓ [39/503] APO    -> FULL HOLD + ADD      $151.34
✓ [40/503] AMAT

## Cell 3: Filter and Display Results by Confluence

In [ ]:
# Filter for FULL HOLD + ADD signals
buy_df = filter_buy_signals(results_df, 'FULL HOLD + ADD')

print(f"{'='*80}")
print(f"🎯 FULL HOLD + ADD SIGNALS: {len(buy_df)} stocks")
print(f"{'='*80}\n")

# Breakdown by confluence
print("📊 Breakdown by Confluence:")
confluence_counts = buy_df['confluence'].value_counts()
for conf, count in confluence_counts.items():
    print(f"  {conf:12s}: {count:3d} stocks")

# Save to workspace scanner_results folder
results_dir = Path.cwd() / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
excel_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
pdf_path = results_dir / f'scanner_report_{timestamp}.pdf'

# Generate Excel
create_excel_output(buy_df, excel_path)
print(f"\n✓ Excel saved to: {excel_path}")

# Generate PDF
print(f"\n📄 Creating PDF research document...")
create_pdf_report(buy_df, results_df, pdf_path, timestamp)
print(f"✓ PDF saved to: {pdf_path}")

# Cleanup old scans (keep 7 most recent, archive rest)
print("\n📁 Managing scan history...")
cleanup_old_scans(results_dir, max_files=7)

🎯 FULL HOLD + ADD SIGNALS: 202 stocks

📊 Breakdown by Confluence:
  WEAK        : 137 stocks
  EXTENDED    :  42 stocks
  BALANCED    :  23 stocks
✓ Excel file created: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_1945.xlsx
  - All: 202 stocks
  - Balanced: 23 stocks
  - Extended: 42 stocks
  - Weak: 137 stocks

✓ Results saved to: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_1945.xlsx

📁 Managing scan history...


## Cell 4: Display BALANCED Stocks (Priority Buy List)

In [4]:
# Filter for BALANCED confluence only
balanced_df = buy_df[buy_df['confluence'] == 'BALANCED'].copy()
balanced_df = balanced_df.sort_values('ticker')

print(f"🎯 BALANCED STOCKS (Priority Buy Candidates): {len(balanced_df)}\n")

if not balanced_df.empty:
    # Display key columns
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200', 'w10', 'w200']
    available_cols = [col for col in display_cols if col in balanced_df.columns]
    
    print(balanced_df[available_cols].to_string(index=False))
    
    print(f"\n✅ These {len(balanced_df)} stocks have:")
    print("  ✓ FULL HOLD + ADD signal (Weekly P1 + Daily P1)")
    print("  ✓ BALANCED confluence (healthy technical setup)")
    print("  ✓ 'Enter on Dip' recommendation")
else:
    print("No BALANCED stocks found in this scan.")

🎯 BALANCED STOCKS (Priority Buy Candidates): 23

ticker  current_price recommendation        d50       d200        w10       w200
  AAPL       267.3900   Enter on Dip 272.943346 231.965436 273.322119 192.079822
   ALL       205.7500   Enter on Dip 204.374413 200.653223 207.079797 152.024460
  AMGN       319.3300   Enter on Dip 323.183571 294.272593 328.780725 261.671647
  CBOE       252.1700   Enter on Dip 251.717000 236.586874 253.522002 172.131055
   CEG       351.0500   Enter on Dip 360.359692 312.646408 354.315369 173.893209
  CINF       162.8000   Enter on Dip 162.212434 150.712410 163.980283 118.920830
   COR       339.7750   Enter on Dip 348.422795 306.013406 350.234854 215.510328
  CSCO        75.7439   Enter on Dip  75.464507  67.039440  76.256867  51.473201
  CTRA        25.4600   Enter on Dip  25.743994  24.726490  26.194452  24.479916
    EW        84.1700   Enter on Dip  84.378200  78.079450  85.277000  81.407450
   EXE       104.9500   Enter on Dip 112.445908 106.291680 1

## Cell 5: Display EXTENDED Stocks (Watch List)

In [ ]:
# Filter for EXTENDED confluence
extended_df = buy_df[buy_df['confluence'] == 'EXTENDED'].copy()
extended_df = extended_df.sort_values('ticker')

print(f"⏳ EXTENDED STOCKS (Wait for Pullback): {len(extended_df)}\n")

if not extended_df.empty:
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200']
    available_cols = [col for col in display_cols if col in extended_df.columns]
    
    # Show first 20
    print(extended_df[available_cols].head(20).to_string(index=False))
    
    if len(extended_df) > 20:
        print(f"\n... and {len(extended_df) - 20} more (see Excel file)")
    
    print(f"\n⚠️ These stocks have strong signals but are technically overbought.")
    print("  → Wait for pullback to support before entering")
else:
    print("No EXTENDED stocks found.")

## Cell 6: Compare with Current Holdings (stocks.txt)

In [5]:
from technical_analysis import parse_stocks_file

# Load current holdings
stocks_file = Path.cwd() / 'stocks.txt'
individual_tickers, baskets = parse_stocks_file(stocks_file)

# Find BALANCED stocks NOT in your portfolio
balanced_tickers = set(balanced_df['ticker'].tolist())
current_tickers = set(individual_tickers)
new_opportunities = balanced_tickers - current_tickers

print(f"📋 Current Portfolio: {len(current_tickers)} stocks")
print(f"🎯 BALANCED Opportunities: {len(balanced_tickers)} stocks")
print(f"\n✨ NEW BALANCED Stocks (not in your portfolio): {len(new_opportunities)}\n")

if new_opportunities:
    new_df = balanced_df[balanced_df['ticker'].isin(new_opportunities)].sort_values('ticker')
    display_cols = ['ticker', 'current_price', 'd50', 'd200', 'w10', 'w200']
    available_cols = [col for col in display_cols if col in new_df.columns]
    print(new_df[available_cols].to_string(index=False))
    print(f"\n💡 Consider adding these {len(new_opportunities)} stocks to stocks.txt for detailed analysis")
else:
    print("All BALANCED opportunities are already in your portfolio!")

# Check which current holdings are also BALANCED
holdings_balanced = balanced_tickers & current_tickers
print(f"\n✅ Current holdings that are BALANCED: {len(holdings_balanced)}")
if holdings_balanced:
    print(f"   {', '.join(sorted(holdings_balanced))}")

📋 Current Portfolio: 13 stocks
🎯 BALANCED Opportunities: 23 stocks

✨ NEW BALANCED Stocks (not in your portfolio): 22

ticker  current_price        d50       d200        w10       w200
  AAPL       267.3900 272.943346 231.965436 273.322119 192.079822
   ALL       205.7500 204.374413 200.653223 207.079797 152.024460
  AMGN       319.3300 323.183571 294.272593 328.780725 261.671647
  CBOE       252.1700 251.717000 236.586874 253.522002 172.131055
   CEG       351.0500 360.359692 312.646408 354.315369 173.893209
  CINF       162.8000 162.212434 150.712410 163.980283 118.920830
   COR       339.7750 348.422795 306.013406 350.234854 215.510328
  CSCO        75.7439  75.464507  67.039440  76.256867  51.473201
  CTRA        25.4600  25.743994  24.726490  26.194452  24.479916
    EW        84.1700  84.378200  78.079450  85.277000  81.407450
   EXE       104.9500 112.445908 106.291680 113.415422  86.063613
   IBM       295.0350 303.774277 269.972813 302.451880 177.716194
  IDXX       676.8600 6

## Cell 7: Overall Signal Distribution

In [7]:
print("📊 Complete S&P 500 Signal Distribution:\n")
print(results_df['signal'].value_counts().to_string())

print(f"\n\n{'='*80}")
print("Summary")
print(f"{'='*80}")
print(f"Total S&P 500 stocks analyzed: {len(results_df)}")
print(f"FULL HOLD + ADD signals: {len(buy_df)}")
print(f"  ├─ BALANCED (buy now): {len(buy_df[buy_df['confluence'] == 'BALANCED'])}")
print(f"  ├─ EXTENDED (watch): {len(buy_df[buy_df['confluence'] == 'EXTENDED'])}")
print(f"  └─ WEAK (skip): {len(buy_df[buy_df['confluence'] == 'WEAK'])}")
print(f"\nExcel file: {output_path}")

📊 Complete S&P 500 Signal Distribution:

signal
FULL HOLD + ADD       202
HOLD MOST → REDUCE     76
FULL CASH / DEFEND     72
CASH                   55
REDUCE                 36
HOLD                   34
SCALE IN               20
LIGHT / CASH            7


Summary
Total S&P 500 stocks analyzed: 502
FULL HOLD + ADD signals: 202
  ├─ BALANCED (buy now): 23
  ├─ EXTENDED (watch): 42
  └─ WEAK (skip): 137

Excel file: C:\Users\karms\Downloads\sp500_analysis_20260105_1931.xlsx
